In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("quikr_car.csv")
df.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [4]:
df.isnull().sum()

name           0
company        0
year           0
Price          0
kms_driven    52
fuel_type     55
dtype: int64

In [5]:
df.columns

Index(['name', 'company', 'year', 'Price', 'kms_driven', 'fuel_type'], dtype='object')

In [6]:
df = df[df['year'].str.isnumeric()]

In [7]:
df['year'] = df['year'].astype(int)

In [8]:
df = df[df['Price'] != "Ask For Price"]

In [9]:
df['Price'] = df['Price'].str.replace(',','').astype(int)

In [10]:
df['kms_driven'] = df['kms_driven'].str.split(' ').str.get(0).str.replace(',','')

In [11]:
df = df[df['kms_driven'].str.isnumeric()]

In [12]:
df['kms_driven'] = df['kms_driven'].astype(int)

In [13]:
df = df[~df['fuel_type'].isnull()]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 816 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        816 non-null    object
 1   company     816 non-null    object
 2   year        816 non-null    int32 
 3   Price       816 non-null    int32 
 4   kms_driven  816 non-null    int32 
 5   fuel_type   816 non-null    object
dtypes: int32(3), object(3)
memory usage: 35.1+ KB


In [15]:
df['name'] = df['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [16]:
df = df.reset_index(drop=True)

In [17]:
df.describe()

,year,Price,kms_driven
count,816.000000,8.160000e+02,816.000000
mean,2012.444853,4.117176e+05,46275.531863
std,4.002992,4.751844e+05,34297.428044
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.912500e+05,56818.500000
max,2019.000000,8.500003e+06,400000.000000


In [18]:
df = df[df['Price'] < 6e6].reset_index(drop=True)

In [19]:
df.to_csv('cleaned.csv')

In [20]:
df.columns

Index(['name', 'company', 'year', 'Price', 'kms_driven', 'fuel_type'], dtype='object')

In [21]:
uniques = []
list1 = ['name', 'company', 'year', 'Price', 'kms_driven', 'fuel_type']
for i in list1:
    unique = df[i].unique()
    uniques.append(unique)

In [22]:
unique_df = pd.DataFrame(uniques).T
unique_df.columns = ['name', 'company', 'year', 'Price', 'kms_driven', 'fuel_type']

In [23]:
unique_df.to_csv("unique_values.csv", index=False)

### **Create Model**

In [24]:
X = df.drop(columns='Price')
y = df['Price']

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

In [28]:
encoder = OneHotEncoder()

In [29]:
encoder.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [30]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [31]:
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'), ['name','company','fuel_type']),
                                        remainder='passthrough')

In [32]:
model = LinearRegression()

In [33]:
pipe = make_pipeline(column_trans, model)

In [34]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [35]:
y_pred = pipe.predict(X_test)

In [36]:
r2_score(y_test, y_pred)

0.44710907399906463

In [37]:
r2_max = 0
best_i = 0
for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    model = LinearRegression()
    pipe = make_pipeline(column_trans, model)
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    if r2 > r2_max:
        r2_max = r2
        best_i = i
print(f"Best R2 score: {r2_max} at random state: {best_i}")

Best R2 score: 0.8457046439268315 at random state: 433


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=best_i)
model = LinearRegression()
pipe = make_pipeline(column_trans, model)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
r2_score(y_test, y_pred)

0.8457046439268315

In [39]:
import pickle

In [40]:
pickle.dump(pipe, open("LinearRegressionModel.pkl", 'wb'))

In [41]:
new_data = pd.DataFrame({'name': ['Maruti Suzuki Swift'], 'company': ['Maruti'], 'year':[2019], 'kms_driven': [100], 'fuel_type': ['Petrol']})
pipe.predict(new_data)

array([458899.18969522])